In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from torch import nn
import matplotlib.pyplot as plt
import math
import ast
import csv



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/Transformer\ Tennis/

# CHECKPOINT_DIR = '/content/drive/MyDrive/Transformer Tennis/checkpoints/'
# os.makedirs(CHECKPOINT_DIR, exist_ok=True)
%load_ext cudf.pandas

Mounted at /content/drive
/content/drive/MyDrive/Transformer Tennis


In [ ]:
! git clone https://github.com/JeffSackmann/tennis_MatchChartingProject.git
! git clone https://github.com/JeffSackmann/tennis_atp.git

fatal: destination path 'tennis_MatchChartingProject' already exists and is not an empty directory.
fatal: destination path 'tennis_atp' already exists and is not an empty directory.


For Bowman

In [ ]:
base_drive_path = '/content/drive/MyDrive/Transformer Tennis/'

# Define the paths to the cloned repositories within your Drive
match_charting_repo_path = os.path.join(base_drive_path, 'tennis_MatchChartingProject')
atp_repo_path = os.path.join(base_drive_path, 'tennis_atp')

points2020_df = pd.read_csv(os.path.join(match_charting_repo_path, 'charting-m-points-2020s.csv'))
points2010_df = pd.read_csv(os.path.join(match_charting_repo_path, 'charting-m-points-2010s.csv'))
points2000s_df = pd.read_csv(os.path.join(match_charting_repo_path, 'charting-m-points-to-2009.csv'))
matches_df = pd.read_csv(os.path.join(match_charting_repo_path, 'charting-m-matches.csv'))
names_df = pd.read_csv(os.path.join(atp_repo_path, 'atp_players.csv'))
rankings20s_df = pd.read_csv(os.path.join(atp_repo_path, 'atp_rankings_20s.csv'))
rankings10s_df = pd.read_csv(os.path.join(atp_repo_path, 'atp_rankings_10s.csv'))
rankings00s_df = pd.read_csv(os.path.join(atp_repo_path, 'atp_rankings_00s.csv'))


<ipython-input-4-3946850075>:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  points2020_df = pd.read_csv(os.path.join(match_charting_repo_path, 'charting-m-points-2020s.csv'))


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

For Conor

In [ ]:
points2020_df = pd.read_csv('tennis_MatchChartingProject/charting-m-points-2020s.csv')
points2010_df = pd.read_csv('tennis_MatchChartingProject/charting-m-points-2010s.csv')
points2000s_df = pd.read_csv('tennis_MatchChartingProject/charting-m-points-to-2009.csv')
matches_df = pd.read_csv('tennis_MatchChartingProject/charting-m-matches.csv')
names_df = pd.read_csv('tennis_atp/atp_players.csv')
rankings20s_df = pd.read_csv('tennis_atp/atp_rankings_20s.csv')
rankings10s_df = pd.read_csv('tennis_atp/atp_rankings_10s.csv')
rankings00s_df = pd.read_csv('tennis_atp/atp_rankings_00s.csv')

<ipython-input-8-1145617803>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  points2020_df = pd.read_csv('tennis_MatchChartingProject/charting-m-points-2020s.csv')
<ipython-input-8-1145617803>:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  names_df = pd.read_csv('tennis_atp/atp_players.csv')


# Feature Engineering

In [ ]:
# Function to encode one match's point sequence with extended context
def process_match(match_id, match_data, score_to_idx_map):
    # Keep only relevant columns and drop rows without score
    match_data = match_data[["Svr", "Pts", "1st", "2nd", "PtWinner", "Gm1", "Gm2", "Set1", "Set2"]].copy()
    match_data = match_data[match_data["Pts"].notna()]

    # Encode features
    match_data["score_encoded"] = match_data["Pts"].map(score_to_idx_map)
    match_data["has_1st_serve"] = match_data["1st"].notna().astype(int)
    match_data["has_2nd_serve"] = match_data["2nd"].notna().astype(int)
    match_data["server_encoded"] = match_data["Svr"].apply(lambda x: 0 if x == 1 else 1)

    # Fill NaNs in game/set columns with 0
    match_data["Gm1"] = match_data["Gm1"].fillna(0).astype(int)
    match_data["Gm2"] = match_data["Gm2"].fillna(0).astype(int)
    match_data["Set1"] = match_data["Set1"].fillna(0).astype(int)
    match_data["Set2"] = match_data["Set2"].fillna(0).astype(int)

    # Create the point-level feature vectors
    feature_cols = ["server_encoded", "score_encoded", "has_1st_serve", "has_2nd_serve", "Gm1", "Gm2", "Set1", "Set2"]
    point_sequence = match_data[feature_cols].values.tolist()

    # Determine match winner (based on majority of points won)
    pt_counts = match_data["PtWinner"].value_counts()
    match_winner = 0 if pt_counts.get(1, 0) > pt_counts.get(2, 0) else 1

    return {
        "match_id": match_id,
        "point_sequence": point_sequence,
        "match_winner": match_winner
    }

# List of dataframes to process
dataframes_to_process = [points2020_df, points2010_df, points2000s_df]
processed_dataframes = []

for df in dataframes_to_process:
    # Group by match_id
    grouped_matches = df.groupby("match_id")

    # Use a smaller sample to avoid long processing times (e.g., first 5 matches)
    sample_match_ids = df["match_id"].drop_duplicates()

    # Create a score-to-index mapping
    all_scores = df["Pts"].dropna().unique()
    score_to_idx = {score: i for i, score in enumerate(all_scores)}

    # Process all sample matches
    processed = [process_match(mid, grouped_matches.get_group(mid), score_to_idx) for mid in sample_match_ids]

    # Convert to a match-level DataFrame
    processed_df = pd.DataFrame(processed)

    processed_df['match_id'] = processed_df['match_id'].astype(str)

    processed_dataframes.append(processed_df)

# Concatenate the processed dataframes if needed
# all_processed_df = pd.concat(processed_dataframes, ignore_index=True)

# Assign the processed dataframes back to their original variable names if desired
points2020s_processed = processed_dataframes[0]
points2010s_processed = processed_dataframes[1]
points2000s_processed = processed_dataframes[2]

all_points_processed = pd.concat([points2020s_processed, points2010s_processed, points2000s_processed], ignore_index=True)
all_points_processed.head()

,match_id,point_sequence,match_winner
0,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,"[[0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, ...",0
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,"[[0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, ...",1
2,20250608-M-Roland_Garros-F-Jannik_Sinner-Carlo...,"[[0, 2, 1, 0, 4, 2, 1, 0], [0, 11, 1, 1, 4, 2,...",0
3,20250606-M-Roland_Garros-SF-Novak_Djokovic-Jan...,"[[1, 11, 1, 1, 4, 5, 0, 1], [1, 12, 1, 1, 4, 5...",1
4,20250604-M-Roland_Garros-QF-Novak_Djokovic-Ale...,"[[0, 0, 1, 1, 0, 0, 0, 0], [0, 9, 1, 1, 0, 0, ...",0


In [ ]:
features_df = all_points_processed.join(matches_df.set_index('match_id'), on='match_id')
features_df = features_df.drop(columns = ["Tournament", "Time", "Court", "Umpire", "Charted by", "Final TB?"])
features_df['match_date'] = features_df['match_id'].str.extract(r'(\d{8})')
features_df['match_date'] = pd.to_datetime(features_df['match_date'], format='%Y%m%d')

# Normalize player names
features_df['Player 1'] = features_df['Player 1'].str.lower().str.strip()
features_df['Player 2'] = features_df['Player 2'].str.lower().str.strip()

names_df['full_name'] = (names_df['name_first'] + ' ' + names_df['name_last']).str.lower().str.strip()
names_df.head()

features_df = features_df.merge(names_df[['player_id', 'full_name']], how='left',
                               left_on='Player 1', right_on='full_name')
features_df = features_df.rename(columns={'player_id': 'player1_id'})
features_df = features_df.merge(names_df[['player_id', 'full_name']], how='left',
                               left_on='Player 2', right_on='full_name')
features_df = features_df.rename(columns={'player_id': 'player2_id'})

features_df.head()

,match_id,point_sequence,match_winner,Player 1,Player 2,Pl 1 hand,Pl 2 hand,Date,Round,Surface,Best of,match_date,player1_id,full_name_x,player2_id,full_name_y
0,20250610-M-ITF_Martos-Q2-Preston_Stearns-Aleja...,"[[0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, ...",0,preston stearns,alejandro lopez escribano,L,L,20250610,Q2,Hard,3,2025-06-10,210475.0,preston stearns,213075.0,alejandro lopez escribano
1,20250609-M-Stuttgart-R32-Benjamin_Bonzi-Jiri_L...,"[[0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, ...",1,benjamin bonzi,jiri lehecka,R,R,20250609,R32,Grass,3,2025-06-09,126127.0,benjamin bonzi,208103.0,jiri lehecka
2,20250608-M-Roland_Garros-F-Jannik_Sinner-Carlo...,"[[0, 2, 1, 0, 4, 2, 1, 0], [0, 11, 1, 1, 4, 2,...",0,jannik sinner,carlos alcaraz,R,R,20250608,F,Clay,5,2025-06-08,206173.0,jannik sinner,207989.0,carlos alcaraz
3,20250606-M-Roland_Garros-SF-Novak_Djokovic-Jan...,"[[1, 11, 1, 1, 4, 5, 0, 1], [1, 12, 1, 1, 4, 5...",1,novak djokovic,jannik sinner,R,R,20250606,SF,Clay,5,2025-06-06,104925.0,novak djokovic,206173.0,jannik sinner
4,20250604-M-Roland_Garros-QF-Novak_Djokovic-Ale...,"[[0, 0, 1, 1, 0, 0, 0, 0], [0, 9, 1, 1, 0, 0, ...",0,novak djokovic,alexander zverev,R,R,20250604,QF,Clay,5,2025-06-04,104925.0,novak djokovic,100644.0,alexander zverev


In [ ]:
def get_latest_prior_ranking(player_id, match_date, df_rankings):
    rankings = df_rankings[(df_rankings['player'] == player_id) &
                           (df_rankings['ranking_date'] <= match_date)]
    if rankings.empty:
        return None
    return rankings.sort_values('ranking_date', ascending=False).iloc[0]['rank']

# create rankings for all rank data
rankings_df = pd.concat([rankings20s_df, rankings10s_df, rankings00s_df], ignore_index=True)

rankings_df['ranking_date'] = pd.to_datetime(rankings_df['ranking_date'], format='%Y%m%d')
features_df['rank_p1'] = features_df.apply(
    lambda row: get_latest_prior_ranking(row['player1_id'], row['match_date'], rankings_df), axis=1)

features_df['rank_p2'] = features_df.apply(
    lambda row: get_latest_prior_ranking(row['player2_id'], row['match_date'], rankings_df), axis=1)

In [ ]:
final_features_df = (features_df.drop(columns = [
    "Player 1", "Player 2", "full_name_x", "full_name_y", "player1_id", "player2_id", "match_date", "Date", "match_id"]))
final_features_df.head()

,point_sequence,match_winner,Pl 1 hand,Pl 2 hand,Round,Surface,Best of,rank_p1,rank_p2
0,"[[0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, ...",0,L,L,Q2,Hard,3,1380.0,NaN
1,"[[0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, ...",1,R,R,R32,Grass,3,73.0,31.0
2,"[[0, 2, 1, 0, 4, 2, 1, 0], [0, 11, 1, 1, 4, 2,...",0,R,R,F,Clay,5,4.0,2.0
3,"[[1, 11, 1, 1, 4, 5, 0, 1], [1, 12, 1, 1, 4, 5...",1,R,R,SF,Clay,5,1.0,4.0
4,"[[0, 0, 1, 1, 0, 0, 0, 0], [0, 9, 1, 1, 0, 0, ...",0,R,R,QF,Clay,5,1.0,7.0


In [ ]:
# I think we can  fill empty spaces with unknown cause there's only one row missing it
final_features_df[['Pl 1 hand', 'Pl 2 hand', 'Round', 'Surface']] = final_features_df[['Pl 1 hand', 'Pl 2 hand', 'Round', 'Surface']].fillna('Unknown')
categorical_cols = ['Pl 1 hand', 'Pl 2 hand', 'Round', 'Surface']
enconded_df = pd.get_dummies(final_features_df, columns=categorical_cols, dtype=int)
enconded_df = enconded_df[enconded_df["Pl 1 hand_20240915"] != 1].drop(columns = ["Pl 1 hand_20240915"]) # This works too
#should we do this? I don't think so cause once it's one hot-encoded filling with zeros actually means something.
# I made changes above to fix this
# enconded_df.fillna(0, inplace=True)

In [ ]:
encoded_df = (enconded_df[["point_sequence", "match_winner", "Best of", "rank_p1", "rank_p2", "Pl 1 hand_L", "Pl 1 hand_R", "Pl 2 hand_L", "Pl 2 hand_R"
, 'Round_BR', 'Round_F', 'Round_PO', 'Round_Q1',
       'Round_Q2', 'Round_Q3', 'Round_QF', 'Round_R128', 'Round_R16',
       'Round_R32', 'Round_R64', 'Round_RR', 'Round_SF', 'Surface_3', 'Surface_Clay', 'Surface_Grass', 'Surface_Hard']])
encoded_df.head()

,point_sequence,match_winner,Best of,rank_p1,rank_p2,Pl 1 hand_L,Pl 1 hand_R,Pl 2 hand_L,Pl 2 hand_R,Round_BR,...,Round_R128,Round_R16,Round_R32,Round_R64,Round_RR,Round_SF,Surface_3,Surface_Clay,Surface_Grass,Surface_Hard
0,"[[0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, ...",0,3,1380.0,NaN,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"[[0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, ...",1,3,73.0,31.0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
2,"[[0, 2, 1, 0, 4, 2, 1, 0], [0, 11, 1, 1, 4, 2,...",0,5,4.0,2.0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,"[[1, 11, 1, 1, 4, 5, 0, 1], [1, 12, 1, 1, 4, 5...",1,5,1.0,4.0,0,1,0,1,0,...,0,0,0,0,0,1,0,1,0,0
4,"[[0, 0, 1, 1, 0, 0, 0, 0], [0, 9, 1, 1, 0, 0, ...",0,5,1.0,7.0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
exclude_cols = ['point_sequence', 'match_winner']

# Get the match-level feature matrix
match_features = encoded_df.drop(columns=exclude_cols)

# Convert to float32 tensor
match_features_tensor = torch.tensor(match_features.astype('float32').values)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# right after you finish cleaning / one-hot encoding...
encoded_df = encoded_df.reset_index(drop=True)

# now when you drop point_sequence & match_winner…
match_features = encoded_df.drop(columns=['point_sequence','match_winner'])
match_features_tensor = torch.tensor(match_features.astype('float32').values)

# lengths should now match:
assert len(match_features_tensor) == len(encoded_df)

# and your loop will never run off the end:
point_match_tensors = []
for i, row in encoded_df.iterrows():
    point_seq = torch.tensor(row['point_sequence'], dtype=torch.float32)
    match_feat = match_features_tensor[i]     # now safe
    match_feat_rep = match_feat.unsqueeze(0).repeat(point_seq.size(0),1)
    point_match_tensors.append(torch.cat([point_seq, match_feat_rep], dim=1))

labels = torch.tensor(encoded_df['match_winner'].values, dtype=torch.float32)

In [ ]:
torch.save(point_match_tensors, 'point_match_tensors.pt')
torch.save(labels, 'labels.pt')

# Model Training

In [ ]:
point_match_tensors_load = torch.load('point_match_tensors.pt')
labels_load = torch.load('labels.pt')

In [ ]:
import torch
from torch.utils.data import Dataset
import random

class FixedPrefixDataset(Dataset):
    def __init__(self, match_tensors, labels,
                 min_prefix=10, prefixes_per_match=20):
        """
        match_tensors: list of [L_i, feat_dim] tensors
        labels:        list or 1D tensor of 0/1
        min_prefix:    smallest t you’ll ever pick
        prefixes_per_match: how many random prefixes you want per match each epoch
        """
        self.match_tensors = match_tensors
        self.labels = labels
        self.min_prefix = min_prefix
        self.p = prefixes_per_match
        self.M = len(match_tensors)

    def __len__(self):
        # total samples per epoch = matches × prefixes_per_match
        return self.M * self.p

    def __getitem__(self, idx):
        # determine which match this is
        match_idx = idx // self.p
        seq       = self.match_tensors[match_idx]
        L         = seq.size(0)

        # sample a random prefix length between min_prefix and L
        t = random.randint(self.min_prefix, L)

        # slice the prefix and grab the label
        prefix = seq[:t]                   # shape [t, feat_dim]
        label  = self.labels[match_idx]    # 0 or 1

        return prefix, label
def collate_fn(batch):
    sequences, labels = zip(*batch)
    # pad sequences to the same length
    padded = pad_sequence(sequences, batch_first=True)   # [B, max_seq_len, feat_dim]
    labels = torch.tensor(labels, dtype=torch.float32)   # [B]
    return padded, labels

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))  # [1, max_len, d_model]

    def forward(self, x):
        # x: [B, L, D]
        return x + self.pe[:, :x.size(1)]

In [ ]:
class MatchOutcomeTransformer(nn.Module):
    def __init__(self, input_dim, d_model=128, nhead=4, num_layers=2, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_enc     = PositionalEncoding(d_model)
        encoder_layer    = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=d_model*4, dropout=dropout)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.classifier  = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: [B, L, input_dim]
        x = self.input_proj(x)       # → [B, L, d_model]
        x = self.pos_enc(x)          # add positional info
        x = x.transpose(0,1)         # → [L, B, d_model]
        x = self.transformer(x)      # → [L, B, d_model]
        x = x.mean(dim=0)            # → [B, d_model]  (mean-pool over points)
        return self.classifier(x).squeeze(1)  # → [B]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

full_dataset = FixedPrefixDataset(
    match_tensors=point_match_tensors,
    labels=labels_load,
    min_prefix=10,
    prefixes_per_match=50
)


# ─── 2) Split into train / val ──────────────────────────────────────────
total = len(full_dataset)
n_val = int(0.2 * total)        # 20% for validation
n_train = total - n_val

train_ds, val_ds = random_split(full_dataset, [n_train, n_val],
                                generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,
                          collate_fn=collate_fn, num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False,
                          collate_fn=collate_fn, num_workers=0)

# ─── 3) Set up model, loss, optimizer ───────────────────────────────────
print("point_match_tensors[0].shape:", point_match_tensors[0].shape) #debug line
model = MatchOutcomeTransformer(input_dim=point_match_tensors[0].size(1),
                                  d_model = 128,
                                  nhead=4,
                                  num_layers=2,
                                  dropout=0.1).to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# ─── 4) Training loop with loss tracking ─────────────────────────────────
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(1, 21):   # e.g. 20 epochs
    # — Train —
    model.train()
    total_train_loss = 0.0
    correct_train = 0
    total_train = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        preds = model(X)
        print("y.shape:", y.shape)
        print("y dtype:", y.dtype)
        print("y unique values:", torch.unique(y))
        print("preds.shape:", preds.shape)
        loss  = criterion(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * X.size(0)
        predicted = (preds > 0.5).float()
        correct_train += (predicted == y).sum().item()
        total_train += y.size(0)

    train_losses.append(total_train_loss / n_train)
    train_acc = correct_train / total_train
    train_accuracies.append(train_acc)

    # — Validate —
    model.eval()
    total_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            preds = model(X)
            loss  = criterion(preds, y)
            total_val_loss += loss.item() * X.size(0)
            predicted = (preds > 0.5).float()
            correct_val += (predicted == y).sum().item()
            total_val += y.size(0)
    val_losses.append(total_val_loss / n_val)
    val_acc = correct_val / total_val
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch:02d}  Train: {train_losses[-1]:.4f}  Val: {val_losses[-1]:.4f}" f"Train Acc: {train_acc:.3f}  Val Acc: {val_acc:.3f}")

# ─── 5) Plot the curves ────────────────────────────────────────────────────
plt.figure(figsize=(6,4))
plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
plt.plot(range(1, len(val_losses)+1),   val_losses,   label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs. Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(6,4)) # New figure for accuracy
plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, len(val_accuracies)+1),   val_accuracies,   label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training vs. Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

point_match_tensors[0].shape: torch.Size([134, 32])


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

full_dataset = FixedPrefixDataset(
    match_tensors=point_match_tensors_load,
    labels=labels_load,
    min_prefix=10,
    prefixes_per_match=50
)


# ─── 2) Split into train / val ──────────────────────────────────────────
total = len(full_dataset)
n_val = int(0.2 * total)        # 20% for validation
n_train = total - n_val

train_ds, val_ds = random_split(full_dataset, [n_train, n_val],
                                generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,
                          collate_fn=collate_fn, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False,
                          collate_fn=collate_fn, num_workers=2)


# Define hyperparameter grid
param_grid = {
    'lr': [1e-4],
    'dropout': [0.1, 0.3],
    'd_model': [128, 256],
    'num_layers': [2, 4, 6],
    'nhead' : [2, 4] }

best_val_acc = 0.0
best_params = None

param_combos = list(itertools.product(*param_grid.values()))

csv_header = ['lr', 'dropout', 'd_model', 'num_layers', 'nhead',
              'final_train_loss', 'final_val_loss', 'final_val_accuracy']

RESULTS_CSV_PATH = '/content/drive/MyDrive/Transformer Tennis/sweep_results.csv'

if not os.path.exists(RESULTS_CSV_PATH):
    with open(RESULTS_CSV_PATH, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(csv_header)

completed_combos_from_csv = set()

if os.path.exists(RESULTS_CSV_PATH):
    with open(RESULTS_CSV_PATH, 'r', newline='') as f:
        reader = csv.reader(f)
        next(reader) # Skip header
        for row in reader:
            # Extract combo parameters from the row (first len(param_grid) columns)
            # Need to convert back to appropriate types if necessary,
            # but for simple types like float/int, they will be strings in CSV.
            # Convert them back to their original types for comparison.
            try:
                combo_params_str = tuple(row[:len(param_grid)])
                # Convert string values to their original types for accurate comparison with the combo tuple
                typed_combo_params = (
                    float(combo_params_str[0]), # lr is float
                    float(combo_params_str[1]), # dropout is float
                    int(combo_params_str[2]),   # d_model is int
                    int(combo_params_str[3]),   # num_layers is int
                    int(combo_params_str[4])    # nhead is int
                )
                completed_combos_from_csv.add(typed_combo_params)
            except (ValueError, IndexError) as e:
                print(f"Warning: Could not parse row from CSV: {row}. Error: {e}")
                continue # Skip this row if parsing fails

CHECKPOINT_DIR = '/content/drive/MyDrive/Transformer Tennis/checkpoints/'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


for i, combo in enumerate(param_combos):
    lr, dropout, d_model, num_layers, nhead = combo
    print(f"\n Sweep {i+1}/{len(param_combos)}: lr={lr}, dropout={dropout}, d_model={d_model}, num_layers={num_layers}, nhead={nhead}")

    # 3) Build model & optimizer
    model = MatchOutcomeTransformer(
        input_dim=point_match_tensors_load[0].size(1),
        d_model=d_model, nhead=nhead,
        num_layers=num_layers, dropout=dropout
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.BCELoss()

    # 4) If we have a checkpoint, load it

    train_losses_epoch = []
    val_losses_epoch = []
    val_accuracies_epoch = []


    # 5) Training loop from start_epoch to max_epoch
    for epoch in range(1, 9):   # if 8 epochs total
        model.train()
        total_train_loss = 0.0
        correct_train = 0
        total_train = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            preds = model(X)
            loss  = criterion(preds, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item() * X.size(0)
            predicted = (preds > 0.5).float()
            correct_train += (predicted == y).sum().item()
            total_train += y.size(0)
        avg_train_loss = total_train_loss / n_train
        train_losses_epoch.append(avg_train_loss)


        # Validation...
        model.eval()
        total_val_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                predicted = (model(X) > .5).float()
                correct += (predicted == y).sum().item()
                total += y.size(0)
            avg_val_loss = total_val_loss / n_val
            val_losses_epoch.append(avg_val_loss)
            val_acc = correct_val / total_val
            val_accuracies_epoch.append(val_acc)

            print(f"  Epoch {epoch:02d}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, Val Acc = {val_acc:.4f}")


    final_train_loss = train_losses_epoch[-1] if train_losses_epoch else None
    final_val_loss = val_losses_epoch[-1] if val_losses_epoch else None
    final_val_accuracy = val_accuracies_epoch[-1] if val_accuracies_epoch else None

    with open(RESULTS_CSV_PATH, 'a', newline='') as f:
        writer = csv.writer(f)
        row_data = list(combo) + [final_train_loss, final_val_loss, final_val_accuracy]
        writer.writerow(row_data)

    print(f" Completed combo {i+1}/{len(param_combos)}. Results saved to {RESULTS_CSV_PATH}")

    if final_val_accuracy is not None and final_val_accuracy > best_val_acc:
        best_val_acc = final_val_accuracy
        best_params = combo
        print("New best validation accuracy found.")
    # Plot accuracy curve for this run
    plt.figure(figsize=(6, 4)) # New figure for loss
    plt.plot(range(1, len(train_losses_epoch) + 1), train_losses_epoch, label='Train Loss')
    plt.plot(range(1, len(val_losses_epoch) + 1), val_losses_epoch, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f"Loss Curve for Combo {i+1}\nlr={lr}, dropout={dropout}, d_model={d_model}, layers={num_layers}, nhead={nhead}")
    plt.legend()
    plt.grid(True)
    plt.show()

print("\nHyperparameter sweep finished.")
if best_params:
    print(f"Best validation accuracy: {best_val_acc:.4f} with parameters: {best_params}")
else:
    print("No combos were run or completed.")



 Sweep 1/24: lr=0.0001, dropout=0.1, d_model=128, num_layers=2, nhead=2


ZeroDivisionError: division by zero

In [ ]:
import itertools
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import random
import csv
import os
import ast

class FixedPrefixDataset(Dataset):
    def __init__(self, match_tensors, labels,
                 min_prefix=10, prefixes_per_match=20):
        """
        match_tensors: list of [L_i, feat_dim] tensors
        labels:        list or 1D tensor of 0/1
        min_prefix:    smallest t you’ll ever pick
        prefixes_per_match: how many random prefixes you want per match each epoch
        """
        self.match_tensors = match_tensors
        self.labels = labels
        self.min_prefix = min_prefix
        self.p = prefixes_per_match
        self.M = len(match_tensors)

    def __len__(self):
        # total samples per epoch = matches × prefixes_per_match
        return self.M * self.p

    def __getitem__(self, idx):
        # determine which match this is
        match_idx = idx // self.p
        seq       = self.match_tensors[match_idx]
        L         = seq.size(0)

        # sample a random prefix length between min_prefix and L
        t = random.randint(self.min_prefix, L)

        # slice the prefix and grab the label
        prefix = seq[:t]                   # shape [t, feat_dim]
        label  = self.labels[match_idx]    # 0 or 1

        return prefix, label
def collate_fn(batch):
    sequences, labels = zip(*batch)
    # pad sequences to the same length
    padded = pad_sequence(sequences, batch_first=True)   # [B, max_seq_len, feat_dim]
    labels = torch.tensor(labels, dtype=torch.float32)   # [B]
    return padded, labels


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

full_dataset = FixedPrefixDataset(
    match_tensors=point_match_tensors_load,
    labels=labels_load,
    min_prefix=10,
    prefixes_per_match=50
)


# ─── 2) Split into train / val ──────────────────────────────────────────
total = len(full_dataset)
n_val = int(0.2 * total)        # 20% for validation
n_train = total - n_val

train_ds, val_ds = random_split(full_dataset, [n_train, n_val],
                                generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,
                          collate_fn=collate_fn, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False,
                          collate_fn=collate_fn, num_workers=2)


# Define hyperparameter grid
param_grid = {
    'lr': [1e-4],
    'dropout': [0.1, 0.3],
    'd_model': [128, 256],
    'num_layers': [2, 4, 6],
    'nhead' : [2, 4] }

param_combos = list(itertools.product(*param_grid.values()))

SWEEP_PROGRESS_FILE = '/content/drive/MyDrive/Transformer Tennis/sweep_progress.txt'
# Directory to save model checkpoints
CHECKPOINT_DIR = '/content/drive/MyDrive/Transformer Tennis/checkpoints/'
os.makedirs(CHECKPOINT_DIR, exist_ok=True) # Create directory if it doesn't exist

completed_combos = {}
# Load existing progress if the file exists
if os.path.exists(SWEEP_PROGRESS_FILE):
    with open(SWEEP_PROGRESS_FILE, 'r') as f:
        for line in f:
            combo_str, acc_str = line.strip().split(': ')
            completed_combos[ast.literal_eval(combo_str)] = float(acc_str)


best_val_acc = 0.0
best_params = None

# Track validation curves for all runs
for i, combo in enumerate(param_combos):
    lr, dropout, d_model, num_layers, nhead = combo

    if combo in completed_combos:
        print(f"\nSkipping completed combo: {combo} with accuracy {completed_combos[combo]:.4f}")
        continue

    print(f"\n Sweep {i+1}/{len(param_combos)}: lr={lr}, dropout={dropout}, d_model={d_model}, num_layers={num_layers}, nhead={nhead}")

    # Instantiate model and optimizer
    model = MatchOutcomeTransformer(
        input_dim=point_match_tensors_load[0].size(1),
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    val_accuracies = []

    # Train for a few epochs
    for epoch in range(1, 13):
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            preds = model(X)
            loss  = criterion(preds, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                preds = model(X)
                predicted = (preds > 0.5).float()
                correct += (predicted == y).sum().item()
                total += y.size(0)

        val_acc = correct / total
        val_accuracies.append(val_acc)
        print(f"  Epoch {epoch:02d}: Val Acc = {val_acc:.4f}")

        if epoch % 5 == 0: # Save every 5 epochs
          checkpoint_path = os.path.join(CHECKPOINT_DIR, f"combo_{i}_epoch_{epoch}.pth")
          torch.save(model.state_dict(), checkpoint_path)
          print(f"    Saved checkpoint to {checkpoint_path}")
    # Save best combo
    if val_accuracies[-1] > best_val_acc:
        best_val_acc = val_accuracies[-1]
        best_params = combo
        print("New best")

    # Plot accuracy curve for this run
    plt.figure()
    plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Validation Accuracy")
    plt.title(f"Val Accuracy\nlr={lr}, dropout={dropout}, d_model={d_model}, layers={num_layers}")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.show()

NameError: name 'torch' is not defined

In [ ]:
# Choose a path in your mounted Drive (or local folder)
MODEL_PATH = '/content/drive/MyDrive/Transformer Tennis/final_model.pth'

# Save only the model parameters
torch.save(model.state_dict(), MODEL_PATH)

print(f"Model weights saved to {MODEL_PATH}")

Model weights saved to /content/drive/MyDrive/Transformer Tennis/final_model.pth
